## Deduplicação de usuários

Deduplicar usuários de repositórios do Github com base em outras informações de perfil disponíveis no dataset.

In [1]:
import csv
import pandas as pd

### Funções utilitárias

* **firstLastName**
    * Extrai primeiro e último nome de um usuário a partir de uma string

In [2]:
def firstLastName(name):
    if name == '\\N':
        return ''
    else:
        try:
            splitName = name.strip().split(' ')
            if len(splitName) > 1:
                first_last_name = splitName[0] + ' ' + splitName[len(splitName)-1]
            else:
                first_last_name = name
        except:
            return ''
        
        return first_last_name

* **extractFakeUsers**
    * Dado um arquivo de usuários identifica usuários fake e acrescenta numa lista

In [3]:
def extractFakeUsers():
    df = pd.read_csv('data/users.csv', delimiter=',', header=None, names=['user_id', 'login', 'name', 'company', 'location', 
                                                                          'email', 'created_at', 'usr_type', 'fake', 'deleted'])
    df['created_at'] = df['created_at'].map(lambda x: x[:10])
    
    # limpa campos com valor \\N
    null_company = df.index[df['company'] == '\\N']
    df['company'][null_company] = ''
    
    null_location = df.index[df['location'] == '\\N']
    df['location'][null_location] = ''
    
    null_email = df.index[df['email'] == '\\N']
    df['email'][null_email] = ''
    
    # usuário unkown
    unknown_users = df.index[df['name'] == 'unknown']
    df['name'][unknown_users] = ''
    
    # usuário no author
    no_author_users = df.index[df['name'] == '(no author)']
    df['name'][no_author_users] = ''
    
    # primeiro e último nomes
    df['first_last_name'] = df['name'].map(firstLastName)
    
    # nomes vazios
    null_name = df.index[df['name'] == '\\N']
    df['name'][null_name] = ''
    
    # seleciona apenas usuários fake
    df = df[df['fake'] == 1]
    
    return df

In [4]:
def deduplicateUsersByEmail(df_fake_users):
    # agrupamento de usuários por e-mail
    
    fakeUsersDict = {}
    
    for usr_id, email in zip(df_fake_users['user_id'].tolist(), df_fake_users['email'].tolist()):
        # para cada usuário, geraremos um dict por e-mail e armazenaremos apenas o primeiro user_id encontrado (deduplicado)

        if email in fakeUsersDict:
            fakeUsersDict[email][1] = fakeUsersDict[email][1] + 1
        else:
            fakeUsersDict[email] = [usr_id, 1] # user_id, ocorrencias

    return fakeUsersDict

In [5]:
def deduplicateUsersByUsername(df_fake_users):
    # agrupamento de usuários por username/login
    
    fakeUsersDict = {}
    
    for usr_id, username in zip(df_fake_users['user_id'].tolist(), df_fake_users['login'].tolist()):
        # para cada usuário, geraremos um dict por username e armazenaremos apenas o primeiro user_id encontrado (deduplicado)

        if username in fakeUsersDict:
            fakeUsersDict[username][1] = fakeUsersDict[username][1] + 1
        else:
            fakeUsersDict[username] = [usr_id, 1] # user_id, ocorrencias

    return fakeUsersDict

In [6]:
def deduplicateUsersByFirstLastName(df_fake_users):
    # agrupamento de usuários por nome de usuário (primeiro e último)

    fakeUsersDict = {}
    
    for usr_id, first_last_name in zip(df_fake_users['user_id'].tolist(), df_fake_users['first_last_name'].tolist()):
        # para cada usuário, geraremos um dict por primeiro e último nome e armazenaremos apenas o primeiro 
        # user_id encontrado (deduplicado)

        if first_last_name in fakeUsersDict:
            fakeUsersDict[first_last_name][1] = fakeUsersDict[first_last_name][1] + 1
        else:
            fakeUsersDict[first_last_name] = [usr_id, 1] # user_id, ocorrencias

    return fakeUsersDict

### Deduplicação da base de usuários

In [7]:
fake_users = extractFakeUsers()

C:\Users\nater\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\nater\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\nater\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\nater\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarn

In [8]:
print('Quantidade de usuários fake:', len(fake_users))

Quantidade de usuários fake: 808557


#### Deduplicação por e-mail

In [9]:
email_deduplication = deduplicateUsersByEmail(fake_users)

In [10]:
if len(email_deduplication) > 0:
    countDeduplication = 0
    sumOccurrences = 0

    for user in email_deduplication:
        if email_deduplication[user][1] > 1:
            # mais de um usuário por e-mail
            if user != '':
                countDeduplication += 1
                sumOccurrences += email_deduplication[user][1]
    
    print('Quantidade usuários deduplicados:', countDeduplication, '(', (countDeduplication/len(fake_users)), '% )')
    print('Média de', (sumOccurrences/countDeduplication), 'usuários por e-mail')
    print('\nQuantidade usuários e-mail vazio:', email_deduplication[''][1])
else:
    print('Nenhum usuário deduplicado utilizando o e-mail')

Quantidade usuários deduplicados: 1410 ( 0.0017438473725414534 % )
Média de 2.4595744680851066 usuários por e-mail

Quantidade usuários e-mail vazio: 1678


#### Deduplicação por username

In [11]:
username_deduplication = deduplicateUsersByUsername(fake_users)

In [12]:
if len(username_deduplication) > 0:
    countDeduplication = 0
    sumOccurrences = 0

    for user in username_deduplication:
        if username_deduplication[user][1] > 1:
            # mais de um usuário por username
            if user != '':
                countDeduplication += 1
                sumOccurrences += username_deduplication[user][1]
    
    print('Quantidade usuários deduplicados:', countDeduplication, '(', (countDeduplication/len(fake_users)), '% )')
    print('Média de', (0 if countDeduplication == 0 else (sumOccurrences/countDeduplication)), 'usuários por username')
    print('\nQuantidade usuários username vazio:', 0 if '' not in username_deduplication else username_deduplication[''][1])
else:
    print('Nenhum usuário deduplicado utilizando o username')

Quantidade usuários deduplicados: 0 ( 0.0 % )
Média de 0 usuários por username

Quantidade usuários username vazio: 0


#### Deduplicação por primeiro e último nomes

In [13]:
first_last_name_deduplication = deduplicateUsersByFirstLastName(fake_users)

In [14]:
if len(first_last_name_deduplication) > 0:
    countDeduplication = 0
    sumOccurrences = 0

    for user in first_last_name_deduplication:
        if first_last_name_deduplication[user][1] > 1:
            # mais de um usuário por e-mail
            if user != '':
                countDeduplication += 1
                sumOccurrences += first_last_name_deduplication[user][1]
    
    print('Quantidade usuários deduplicados:', countDeduplication, '(', (countDeduplication/len(fake_users)), '% )')
    print('Média de', (sumOccurrences/countDeduplication), 'usuários por primeiro e último nomes')
    print('\nQuantidade usuários nome vazio:', first_last_name_deduplication[''][1])
else:
    print('Nenhum usuário deduplicado utilizando o primeiro e último nomes')

Quantidade usuários deduplicados: 98406 ( 0.12170570534915906 % )
Média de 3.85567953173587 usuários por primeiro e último nomes

Quantidade usuários nome vazio: 33783
